In [ ]:
import gurobipy as gp
from gurobipy import GRB


# Initialise Model
m = gp.Model("4-3-1")


# Initialise Decision Variables
indices = list(range(0, 12))
indices_2 = list(range(0, 6))
x = m.addVars(indices, vtype=GRB.BINARY, name='x')
c = m.addVars(indices, lb = 0, vtype=GRB.CONTINUOUS, name='c') 
z = m.addVars(indices, vtype=GRB.INTEGER, name='z')
y = m.addVars(indices, lb = 0, vtype=GRB.CONTINUOUS, name='y')
beta = m.addVars(indices_2, lb = 0, vtype=GRB.INTEGER, name='Beta') 


# Set Objective function
m.setObjective(y.sum())


# Define the alpha matrix
import itertools   # For creating all unique pairs for the face-up cards (left and right)
card_values = range(1, 5)  # Card values from 1 to 5

alpha_matrix = []
condensed_matrix = []

for left in range(1,5):
    for hidden1, hidden2 in itertools.combinations(card_values, 2):
        if hidden1 != left and hidden2 != left: 
            alpha_i1 = left
            alpha_i2 = 1 if hidden1 > left else 0
            alpha_i3 = 1 if hidden2 > left else 0
            alpha_j4 = hidden1
            alpha_j5 = hidden2
            
            alpha_matrix.append([alpha_i1, alpha_i2, alpha_i3, alpha_j4, alpha_j5]) 

            sorted_row = sorted([alpha_i1,alpha_j4, alpha_j5])
            condensed_matrix.append(sorted_row)


# alpha_matrix
# condensed_matrix


# This is to for ease of coding out the matching constraints (see the variable 'new_matrix')
from collections import defaultdict

# Initialize a dictionary to store the rows grouped by their elements
grouped_rows = defaultdict(list)

# Iterate through the condensed matrix
for index, row in enumerate(condensed_matrix):
    # Convert the row to a tuple to make it hashable and usable as a dictionary key
    key = tuple(row)
    # Append the row index to the list corresponding to the key
    grouped_rows[key].append(index)

# Define the size of the new matrix
num_rows = len(grouped_rows)
num_cols = 3     # Assuming all rows have the same number of columns

# Initialize a new matrix with zeros
new_matrix = [[0] * num_cols for _ in range(num_rows)]

# Iterate through the grouped rows
for row_index, indices in enumerate(grouped_rows.values()):
    # Fill the new matrix with indices
    for col_index, index in enumerate(indices):
        new_matrix[row_index][col_index] = index

# Print the new matrix
for row in new_matrix:
    print(row)



# Create constraints
P = 5

for i in range(12):
    m.addConstr(y[i] + (1 - x[i]) * 5 >= c[i])    

for i in range(4):
    m.addConstr(gp.quicksum(x[new_matrix[i][j]] for j in range(3)) == 1)

for i in range(4):
    m.addConstr(gp.quicksum(x[3*i + j] for j in range(3)) <= 1, 'x_sum_{}'.format(i))

for i in range(12):
    m.addConstr(c[i] == beta[0] + gp.quicksum(beta[k+1] * alpha_matrix[i][k] for k in range(0, 5)) - z[i] * P)


# Additional Cuts
m.addConstr(beta.sum() >= 1)

for i in range(6):
    m.addConstr(beta[i] <= P)  

m.addConstr(beta[4] >= 1)
m.addConstr(beta[5] >= 1)


# Solve the Model
m.setParam('TimeLimit', 5*60)
m.optimize()


# Print the values of the 'c' variables
for i in range(12):
    print('c[{}]: {}'.format(i, c[i].X))


# Initialize a counter for zero
zero_count = 0

for i in range(12):
     if c[i].X == 0.0:
        zero_count += 1

print(zero_count)


# Print the coefficients Beta
for i in range(6):
    print('beta[{}]: {}'.format(i, beta[i].X))


# Print the values of the 'x' variables
for i in range(12):
    # print('x[{}]: {}'.format(i, x[i].X))
    if (x[i].X == 1):
        print('x[{}]: {}'.format(i, x[i].X), 'c[{}]: {}'.format(i, c[i].X))